# Projet 3 : Concevez une application au service de la santé publique #

### Import du nécessaire ###

In [16]:
import numpy as np
import annexe as an
# import plotly.graph_objs as go
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', None)
FICHIER ='F:/cour/OC/projet3/en.openfoodfacts.org.products.csv'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Observons quelques lignes de notre fichier

In [17]:
food_dataframe = pd.read_csv(FICHIER,sep='\t',low_memory=False)
display(food_dataframe.sample(10,random_state = 148625))

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
1386129,7622210763570,http://world-en.openfoodfacts.org/product/7622210763570/toblerone-crispy-coconut-360g,kiliweb,1519635670,2018-02-26T09:01:10Z,1519736122,2018-02-27T12:55:22Z,Toblerone Crispy Coconut 360G,NaN,NaN,NaN,NaN,NaN,NaN,Toblerone,toblerone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-completed, en:ingredients-to-be-completed, en:expiration-date-to-be-completed, en:packaging-code-to-be-completed, en:characteristics-to-be-completed, en:categories-to-be-completed, en:brands-completed, en:packaging-to-be-completed, en:quantity-to-be-completed, en:product-name-completed, en:photos-to-be-validated, en:photos-uploaded","en:to-be-completed,en:nutrition-facts-comple

In [3]:
for column in dataframe.columns:
    print(column)

code
url
creator
created_t
created_datetime
last_modified_t
last_modified_datetime
product_name
abbreviated_product_name
generic_name
quantity
packaging
packaging_tags
packaging_text
brands
brands_tags
categories
categories_tags
categories_en
origins
origins_tags
origins_en
manufacturing_places
manufacturing_places_tags
labels
labels_tags
labels_en
emb_codes
emb_codes_tags
first_packaging_code_geo
cities
cities_tags
purchase_places
stores
countries
countries_tags
countries_en
ingredients_text
allergens
allergens_en
traces
traces_tags
traces_en
serving_size
serving_quantity
no_nutriments
additives_n
additives
additives_tags
additives_en
ingredients_from_palm_oil_n
ingredients_from_palm_oil
ingredients_from_palm_oil_tags
ingredients_that_may_be_from_palm_oil_n
ingredients_that_may_be_from_palm_oil
ingredients_that_may_be_from_palm_oil_tags
nutriscore_score
nutriscore_grade
nova_group
pnns_groups_1
pnns_groups_2
states
states_tags
states_en
brand_owner
ecoscore_score_fr
ecoscore_grade_fr


50% glucides : bon glucide **carbohydrates** on un faible indice glycémique **glycemic-index_100g
** (pain complet, pates de blé dur, légumineuse)
20% lipides (après l'effort pk pas, avant non) acide gras omégas 3 **'alpha-linolenic','eicosapentaenoic-acid','docosahexaenoic-acid' omega-3-fat** et omégas 6 (renforce coeur et circulation) (poisson eau froide saumon maquereau, huile végétal
15% proteines source précieuse de protéines : oeuf, viande maigre, produit laitier et legumineuses
3h entre le gros repas et sport (noix , morceau de choco avant le sport), blanc d'oeuf 2h d'interval, boire du lait écrémé
boire important pour l'acheminement des nutriment (riche en sodium), eau gazeuse plus jus de fruit apres l'effort(1 pour 1)
avant entrainnement: glucides(pates, pain complet), oeuf brouillées,fruit yaourt,pauvre en fibre
pendant celui-ci:eau, boisson isotonique,barre de muesli
apres:jus de pomme petillant avec une pincée de sel,pas de graisse,glucides, protéines de ahute qualité,produit laitier


Activité de moins de 30 min : Il n’est pas nécessaire de boire pendant l’effort
Activité de 30 à 60 min : il faut boire de 125 à 250 ml de liquide à toutes les 15 à 20 minutes.
Activité de 60 min, il ne faut pas attendre d’avoir soif pour boire de l’eau
Activité durant 1h à 3h : il est préférable de choisir une boisson contenant du sucre
Activité de plus de 3 h : préférez une boisson contenant du sucre et un peu de sel (pour le sodium). À noter que le sodium et le potassium dans la boisson de réhydratation ne sont pas indispensables si l’activité physique dure moins de 3 h à 4 h, à moins que la sudation engendrée par l’activité physique ne soit excessive.

In [ ]:
Affichage de quelques lignes après avoir gardé les colonnes pertinentes

In [25]:
dataframe_purify = an.open_csv_usefull_column()
# display(dataframe_purify.sample(10,random_state = 148625))

Voici quelques variables pertinentes pour un application orienté sport

In [28]:
trace_element_variables = ['sodium_100g','zinc_100g','copper_100g','potassium_100g','selenium_100g','manganese_100g','chromium_100g','iodine_100g']
acid_variables = ['omega-3-fat_100g','omega-6-fat_100g','omega-9-fat_100g']
vitamin_variables = ['vitamin-b1_100g','vitamin-b2_100g','vitamin-b6_100g']
sugar_variables = ['carbohydrates_100g','glycemic-index_100g','sugars_100g','-sucrose_100g','-glucose_100g','-fructose_100g','-lactose_100g','-maltose_100g','-maltodextrins_100g']
string_variables = ['pnns_groups_1','pnns_groups_2','nutriscore_score','nutriscore_grade','ecoscore_score_fr','ecoscore_grade_fr']

In [12]:
print("shape ",dataframe_purify.shape)

shape  (1760097, 152)


In [27]:
print("shape ",dataframe_purify.shape)
regex = re.compile(".*_100g.*")

column_list = list(filter(regex.match, dataframe_purify.columns))
# print(column_list)
an.remove_little_quintille(dataframe_purify,column_list)
print("shape ",dataframe_purify.shape)
# for elem in trace_element_variables:
#     print("Element : ",elem)
#     print("quantile 0.99 : ",np.quantile(dataframe_purify[elem].dropna().to_numpy(),0.99))
#     print("quantile 0.01 : ",np.quantile(dataframe_purify[elem].dropna().to_numpy(),0.01))
#     len(dataframe_purify[elem].dropna().to_numpy())

shape  (1704354, 152)
energy-kj_100g
energy-kcal_100g
energy_100g
energy-from-fat_100g
fat_100g
saturated-fat_100g
-butyric-acid_100g
-caproic-acid_100g
-caprylic-acid_100g
-capric-acid_100g
-lauric-acid_100g
-myristic-acid_100g
-palmitic-acid_100g
-stearic-acid_100g
-arachidic-acid_100g
-behenic-acid_100g
-lignoceric-acid_100g
-cerotic-acid_100g
-montanic-acid_100g
-melissic-acid_100g
monounsaturated-fat_100g
polyunsaturated-fat_100g
omega-3-fat_100g
-alpha-linolenic-acid_100g
-eicosapentaenoic-acid_100g
-docosahexaenoic-acid_100g
omega-6-fat_100g
-linoleic-acid_100g
-arachidonic-acid_100g
-gamma-linolenic-acid_100g
-dihomo-gamma-linolenic-acid_100g
omega-9-fat_100g
-oleic-acid_100g
-elaidic-acid_100g
-gondoic-acid_100g
-mead-acid_100g
-erucic-acid_100g
-nervonic-acid_100g
trans-fat_100g
cholesterol_100g
carbohydrates_100g
sugars_100g
-sucrose_100g
-glucose_100g
-fructose_100g
-lactose_100g
-maltose_100g
-maltodextrins_100g
starch_100g
polyols_100g
fiber_100g
-soluble-fiber_100g
-inso

In [30]:
for elem in sugar_variables:
    print("\n\ncolumn : ",elem )
    display("max :",dataframe_purify[elem].max())
    display("min :",dataframe_purify[elem].min())
    display("moyenne :",dataframe_purify[elem].mean())
    display("mediane :",dataframe_purify[elem].median())
    display(dataframe_purify[elem].value_counts().head(10))#.sample(10,random_state = 148625))  



column :  carbohydrates_100g


'max :'

95.65

'min :'

0.0

...

'moyenne :'

2.8302933339393945

'mediane :'

0.12

0.12000     13
0.00012      5
1.50000      3
0.20000      2
14.80000     1
10.30000     1
0.22600      1
0.86000      1
3.40000      1
2.20000      1
Name: -maltodextrins_100g, dtype: int64

In [29]:
for elem in string_variables:
    display(dataframe[elem].value_counts())#.sample(10,random_state = 148625))
# display(dataframe['pnns_groups_2'].value_counts())#.sample(10,random_state = 148625))

unknown                    1076213
Sugary snacks               140583
Fish Meat Eggs               89989
Milk and dairy products      86118
Cereals and potatoes         78642
Fat and sauces               65913
Beverages                    64347
Composite foods              52040
Fruits and vegetables        45786
Salty snacks                 31218
fruits-and-vegetables         7286
sugary-snacks                 5333
cereals-and-potatoes            56
salty-snacks                     6
Name: pnns_groups_1, dtype: int64

unknown                             1076213
Sweets                                66254
Biscuits and cakes                    60330
Dressings and sauces                  46841
Cheese                                42334
One-dish meals                        41115
Cereals                               37578
Processed meat                        34276
Milk and yogurt                       30844
Meat                                  26627
Fish and seafood                      24574
Sweetened beverages                   22234
Vegetables                            21679
Bread                                 21036
Fats                                  19072
Fruits                                15813
Alcoholic beverages                   15582
Chocolate products                    13999
Salty and fatty products              13480
Appetizers                            10687
Unsweetened beverages                  9841
Artificially sweetened beverages       9110
Breakfast cereals               

 14.0    34069
 0.0     32768
 13.0    29551
 11.0    28903
 2.0     27491
 15.0    27170
 12.0    26686
 3.0     26347
 1.0     26266
 16.0    25225
-1.0     24714
 4.0     23705
 5.0     20851
 17.0    19936
 18.0    18346
-4.0     18250
 6.0     18184
 19.0    18168
-2.0     17130
 10.0    16872
 20.0    16292
-3.0     15440
 7.0     13415
 9.0     13292
 21.0    13273
 8.0     12732
 24.0    11379
 22.0    10971
-5.0     10781
 23.0    10582
 25.0    10012
 26.0     9006
-6.0      8863
 27.0     5594
-7.0      3736
-8.0      2553
 28.0     1978
-9.0      1239
-10.0      907
 29.0      600
 30.0      408
-11.0      353
-12.0      256
 31.0      170
-13.0      153
 32.0       88
-14.0       73
 33.0       68
 34.0       57
 35.0       39
 36.0       35
 37.0       10
 40.0        7
-15.0        3
 38.0        2
 39.0        2
Name: nutriscore_score, dtype: int64

d    203743
c    138658
e    123803
a    105482
b     83315
Name: nutriscore_grade, dtype: int64

 39.0     32446
 34.0     13749
 43.0     13520
 9.0      11949
 29.0     11511
 36.0     10673
 67.0     10430
 19.0      8871
 55.0      8189
 41.0      8056
 45.0      8034
 82.0      8005
 79.0      7846
 33.0      7778
 42.0      7733
 35.0      7568
 85.0      7316
 60.0      7139
 32.0      7091
 80.0      6992
 21.0      6634
 18.0      6520
 63.0      6345
 76.0      6208
 59.0      5654
 81.0      5525
 65.0      5378
 83.0      5245
 75.0      5213
 22.0      4965
 54.0      4792
 64.0      4625
 72.0      4378
 74.0      4297
 24.0      4220
 77.0      4178
 30.0      3903
 12.0      3718
 50.0      3682
 78.0      3543
 20.0      3130
 73.0      2993
 51.0      2974
 56.0      2917
 66.0      2906
 6.0       2865
 84.0      2860
 26.0      2857
 52.0      2800
 31.0      2741
 23.0      2611
 49.0      2543
 70.0      2516
 40.0      2483
 27.0      2481
 44.0      2426
-8.0       2409
 53.0      2344
 91.0      2306
 69.0      2207
 10.0      2199
 28.0      2137
-4.0    

d    131993
b    131115
c     86645
e     54049
a     17567
Name: ecoscore_grade_fr, dtype: int64

In [52]:
dataframe_purify[dataframe_purify["sodium_100g"] > 6]


,code,url,product_name,abbreviated_product_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,emb_codes,emb_codes_tags,first_packaging_code_geo,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_quantity,no_nutriments,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
64,0000000005081,http://world-en.openfoodfacts.org/product/0000000005081/salade-verte,Salade verte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,https://static.openfoodfacts.org/images/products/000/000/000/5081/front_fr.4.400.jpg,https://static.openfoodfacts.org/images/products/000/000/000/5081/front_fr.4.200.jpg,https://static.openfoodfacts.org/images/products/000/000/000/5081/ingredients_fr.8.400.jpg,https://static.openfoodfacts.org/images/products/000/000/000/5081/ingredients_fr.8.200.jpg,NaN,NaN,NaN,16.0,67.0,NaN,0.3,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,NaN,NaN,NaN,18.30000,7.320000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,0000000018371,http://world-en.openfoodfacts.org/product/0000000018371/real-salt-granular-redmond,Real Salt Granular,NaN,NaN,NaN,NaN,NaN,Redmond,redmond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,United States,Ancient sea salt with natural trace minerals,NaN,NaN,NaN,NaN,NaN,1.4,NaN,NaN,NaN,unknown,unknown,NaN,NaN,

In [21]:
dataframe_purify[dataframe_purify["selenium_100g"] == 38.8]
dataframe.iloc[1025584]

code                                                                                                                                                                                                                                                                                                                                              3770013130113
url                                                                                                                                                                                                                                              http://world-en.openfoodfacts.org/product/3770013130113/kefir-de-fruits-gingembre-bio-le-labo-dumoulin-lld-cie
product_name                                                                                                                                                                                                                                                                                            

In [26]:
dataframe["carbohydrates_100g"].sort_values().head(100)

1025584   -1.0
625136     0.0
1424501    0.0
1424502    0.0
1144557    0.0
1424510    0.0
336392     0.0
336390     0.0
336388     0.0
1424511    0.0
336385     0.0
1376432    0.0
680279     0.0
336384     0.0
336383     0.0
1007021    0.0
336382     0.0
336373     0.0
336371     0.0
469951     0.0
1424544    0.0
100039     0.0
1424549    0.0
1424550    0.0
1006941    0.0
336349     0.0
336347     0.0
336341     0.0
469897     0.0
1424499    0.0
1424498    0.0
1424497    0.0
100103     0.0
336462     0.0
336459     0.0
336457     0.0
1564861    0.0
336456     0.0
1659933    0.0
336454     0.0
336451     0.0
336450     0.0
336449     0.0
1659926    0.0
336440     0.0
100087     0.0
336439     0.0
336430     0.0
1007124    0.0
1007120    0.0
469873     0.0
1007117    0.0
1424492    0.0
1424493    0.0
336394     0.0
1424494    0.0
1424495    0.0
100073     0.0
1424496    0.0
336339     0.0
336338     0.0
336334     0.0
336332     0.0
470041     0.0
1006883    0.0
336330     0.0
336328    

determiner les quantile avec scipy pour eliminer les valeur aberrantes
faire un pandas.dataframe.corr pour essayer de trouver une corelation entre deux variable (facteur 1 , variable egal, donc redondant et à sortir)
nettoyer pnns a cause de la casse (librairie nltk)
comparer les sucre et les glucide pour avoir l'indice glycemique

In [22]:
dataframe2 = dataframe.drop(dataframe[dataframe["carbohydrates_100g"] > 100].index)
dataframe2 = dataframe2.drop(dataframe2[dataframe2["carbohydrates_100g"] < 0].index)
dataframe2 = dataframe2.drop(dataframe2[dataframe2["sodium_100g"] > 100].index)
dataframe2 = dataframe2.drop(dataframe2[dataframe2["sodium_100g"] < 0].index)

sns.set_theme(style="whitegrid")
f, ax = plt.subplots(figsize=(6, 15))
dataframe2 = dataframe2.sort_values(by=['carbohydrates_100g'],ascending=False)
sns.set_color_codes("pastel")
sns.barplot(x='carbohydrates_100g', y=dataframe2.index, data=dataframe2,label='carbohydrates_100g', color="b")
ax.legend(ncol=1, loc="lower right", frameon=True,fontsize='large')
ax.xaxis.tick_top()    
# if title == "Study years in selected countries":
#     for i,value in enumerate(dataframe2[sort_by]):
#         ax.text(value+3/xmin, i + 0.2,str(value),fontsize=15)
# else:
#     for i,value in enumerate(dataframe2[sort_by]):
#         ax.text(value+3/xmin, i + 0.2,str(int(value)),fontsize=15)    
plt.xlabel( xlabel="title",fontsize=18)
ax.set(xlim=(0, 100), ylabel="", xlabel='title')
sns.despine(left=True, bottom=True)

In [ ]:

# display(dataframe2)#["carbohydrates_100g"].sort_values(ascending=False).head(1))
sns.set_theme(style="ticks")
# diamonds = sns.load_dataset("diamonds")

f, ax = plt.subplots(figsize=(10, 6))
sns.despine(f)

sns.histplot(
    dataframe2,
    x="carbohydrates_100g",#hue="sodium_100g",
    #multiple="stack",
    palette="light:m_r",
    edgecolor="0.3",
    linewidth=0.5,
#     log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
ax.set_xticks([0, 20, 40, 60, 80,100])

In [ ]:
sns.set_theme(style="whitegrid")


# Draw a scatter plot while assigning point colors and sizes to different
# variables in the dataset
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
clarity_ranking = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
sns.scatterplot(x="omega-3-fat_100g", y="omega-9-fat_100g",
                hue="clarity", size="depth",
                palette="ch:r=-.2,d=.3_r",
                hue_order=clarity_ranking,
                sizes=(1, 8), linewidth=0,
                data=dataframe2, ax=ax)